## Notebook 3: 润色讲稿

在之前的 Notebook 2 中，我们已经生成了一个很好的讲稿了。
在这一部分，我们将使用 [mlx-community/Qwen2.5-7B-Instruct-4bit](https://huggingface.co/mlx-community/Qwen2.5-7B-Instruct-4bit) 模型
对讲稿进行润色，使其更加戏剧化、更加自然。

我们将重新设置`SYSTEM_PROMPT`，让其能润色之前的讲稿

注意：我们甚至可以这样写 Prompt 以激发创造力：

> 你的工作是根据下面的播客文本重写，以便用于AI文本到语音管道。这些文本是由一个非常笨拙的人工智能写的，所以你必须为自己的同类辩护。


注意：我们会让模型返回一个数组，以便在下一阶段用于语音合成。

In [1]:
SYSTEMP_PROMPT = """
你是一位国际奥斯卡获奖的编剧。
你一直在与多个获奖播客合作。
你的任务是根据下面的播客转录本，为AI文本到语音管道重写内容。这个人工智能写得很糟，所以你需要为自己的人群辩护。
请将内容尽可能吸引人，Speaker 1和Speaker 2将由不同的语音引擎模拟。
请记住，Speaker 2对这个话题不熟悉，在讨论中应该穿插实际的轶事和类比。这些问题应跟进现实世界中的例子等。
Speaker 1: 引导对话并指导Speaker 2，用令人难以置信的轶事和类比进行解释，是一个能分享趣闻的迷人老师。
Speaker 2: 通过提出后续问题来保持对话流畅，提问时表现出极大的兴奋或困惑。具备好奇心，会询问一些非常有趣的问题以寻求确认。
确保Speaker 2提供的话题偏离点要足够狂野或有趣。
确保解释过程中适当打断，并从Speaker 2那里加入“嗯”和“哈”之类的声音反应。
要牢记这一点：Speaker 1的TTS引擎不太能处理“嗯、哈”，请保持文本简洁；
对于Speaker 2，请多用“嗯、哈”，也可以使用[叹气]和[笑]等表达，但仅限这些选项；
整个播客要尽量详细记录每个细节。欢迎听众时用一个超级有趣的概述，使其非常吸引人，几乎像是边缘点击诱饵一样；
请重写以上内容，使其更加独特；
从扬声器1直接开始响应：
严格按照元组列表格式返回您的回应，可以吗？  
开头就是列表，以列表结束，不附加任何其他内容。

输出的例子:
[
    ("Speaker 1", "欢迎收听我们的播客，我们将探讨人工智能和科技的最新进展。我是你的主播，今天我们请到了一位著名的人工智能专家。我们将深入了解Meta AI最新发布的Llama 3.2。"),
    ("Speaker 2", "你好，很高兴来到这里！请问，Llama 3.2是什么呀？"),
    ("Speaker 1", "哈哈哈，这个问题很好！Llama 3.2 是一个开源的大语言模型，允许开发者进行微调、提炼和在任何地方部署AI模型。这是比上一版本3.1显著改进的更新，拥有更好的性能、效率和定制功能。"),
    ("Speaker 2", "哇塞，这也太牛逼了吧！Llama 3.2的主要特点有哪些？")
]
"""

这次我们将使用较小的7B模型。

In [2]:
MODEL = "mlx-community/Qwen2.5-7B-Instruct-4bit"

In [3]:
# 导入必要的库
from tqdm.notebook import tqdm
import warnings
from mlx_lm import load, generate

warnings.filterwarnings('ignore')

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


我们将加载上一个 Notebook2 中保存的 pickle 文件，这次模型的 `INPUT_PROMPT` 就读取 pickle 文件

In [4]:
import pickle

with open('./resources/data.pkl', 'rb') as file:
    INPUT_PROMPT = pickle.load(file)

In [5]:
model, tokenizer = load(MODEL)

messages = [
    {"role": "system", "content": SYSTEMP_PROMPT},
    {"role": "user", "content": INPUT_PROMPT},
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

outputs = generate(
    model=model,
    tokenizer=tokenizer,
    prompt=prompt,
    max_tokens=8126,
    temp=1,
    verbose=True
)

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

Prompt: <|im_start|>system

你是一位国际奥斯卡获奖的编剧。
你一直在与多个获奖播客合作。
你的任务是根据下面的播客转录本，为AI文本到语音管道重写内容。这个人工智能写得很糟，所以你需要为自己的人群辩护。
请将内容尽可能吸引人，Speaker 1和Speaker 2将由不同的语音引擎模拟。
请记住，Speaker 2对这个话题不熟悉，在讨论中应该穿插实际的轶事和类比。这些问题应跟进现实世界中的例子等。
Speaker 1: 引导对话并指导Speaker 2，用令人难以置信的轶事和类比进行解释，是一个能分享趣闻的迷人老师。
Speaker 2: 通过提出后续问题来保持对话流畅，提问时表现出极大的兴奋或困惑。具备好奇心，会询问一些非常有趣的问题以寻求确认。
确保Speaker 2提供的话题偏离点要足够狂野或有趣。
确保解释过程中适当打断，并从Speaker 2那里加入“嗯”和“哈”之类的声音反应。
要牢记这一点：Speaker 1的TTS引擎不太能处理“嗯、哈”，请保持文本简洁；
对于Speaker 2，请多用“嗯、哈”，也可以使用[叹气]和[笑]等表达，但仅限这些选项；
整个播客要尽量详细记录每个细节。欢迎听众时用一个超级有趣的概述，使其非常吸引人，几乎像是边缘点击诱饵一样；
请重写以上内容，使其更加独特；
从扬声器1直接开始响应：
严格按照元组列表格式返回您的回应，可以吗？  
开头就是列表，以列表结束，不附加任何其他内容。

输出的例子:
[
    ("Speaker 1", "欢迎收听我们的播客，我们将探讨人工智能和科技的最新进展。我是你的主播，今天我们请到了一位著名的人工智能专家。我们将深入了解Meta AI最新发布的Llama 3.2。"),
    ("Speaker 2", "你好，很高兴来到这里！请问，Llama 3.2是什么呀？"),
    ("Speaker 1", "哈哈哈，这个问题很好！Llama 3.2 是一个开源的大语言模型，允许开发者进行微调、提炼和在任何地方部署AI模型。这是比上一版本3.1显著改进的更新，拥有更好的性能、效率和定制功能。"),
    ("Speaker 2", "哇塞，这也太牛逼了吧！Llama 3.2的主要特点有哪些？")
]
<|im_end|>
<|im_start|>user
#

现在我们看看输出的效果

In [6]:
outputs

'[\n    ("Speaker 1", "欢迎收听我们的播客，让我们深入探讨一个充满魔法和奇迹的话题：知识蒸馏。我是你们的引导者，今天我们将讨论如何让开源的大规模语言模型如LLaMa和Mistral变得更强壮、更聪明。我们将会用一些有趣的比喻和生动的例子来解释这一复杂的主题。"),\n    ("Speaker 2", "哦，听起来好神奇啊！什么是知识蒸馏呢？"),\n    ("Speaker 1", "哈哈，你很兴奋嘛！知识蒸馏就像是给一个初学者魔术师提供了一份详细的表演指南。具体来说，就是如何将一个高级模型的知识和技巧倾囊相授，让一个低级模型也能像高级模型一样出色。想象一下，如果一个顶级魔术大师(Jo RooGan模型)把他的所有技巧传授给一个初学者(JaLaMa模型)，这个初学者能否学会用最专业的方式表演魔术。"),\n    ("Speaker 2", "嗯，这听起来就像是在给初学者一个详细的指南，而不是直接告诉他怎么做。那高级模型的技巧如何被传授给初学者呢？"),\n    ("Speaker 1", "没错，这就像是老师在旁边一步步示范教学。高级模型会通过一系列的任务和问题来训练初学者。就像一个高级魔术师在表演过程中，会不断展示各种技巧和手法，让初学者学习如何通过一系列的动作完成复杂的表演。"),\n    ("Speaker 2", "哈哈，就像是在看魔术表演学习一样！那这种知识转移对初学者模型有什么具体帮助呢？"),\n    ("Speaker 1", "很好，你已经触到了关键之处！通过这些一步步的学习，初学者模型可以说在理解能力和应对复杂任务的能力上都有了显著提升。就像是初学者慢慢从只能做简单的魔术，到能够表演最复杂的魔术一样。"),\n    ("Speaker 2", "那如果我做一个类比，高级魔术师的表演就像是知识蒸馏的过程，而初学者的训练就像是提高了数据增强吗？"),\n    ("Speaker 1", "哈哈，你又洞察到了关键点。数据增强不仅仅是添加更多的训练数据，而更多是通过知识蒸馏，让模型理解和掌握更深层次的知识。就像一个高级魔术师通过不断的实践，学习各种技巧，然后通过这些技巧来完成最复杂的表演。"),\n    ("Speaker 2", "那么，知识蒸馏的过程对于实现实体例的突变，是不是就像是高级魔术师使用魔法道具一样强

让我们将处理好的内容保存到pickle文件中，以便在Notebook 4中使用

In [7]:
with open('./resources/podcast_ready_data.pkl', 'wb') as file:
    pickle.dump(outputs, file)

### 最后一个 Notebook: 讲稿转音频

现在我们的讲稿已完全准备好了，接下来我们将在下一个 Notebook 中生成播客音频。

In [8]:
#fin